In [1]:
from scenariogeneration import xosc

In [2]:
MONDEO_ID = "1"
UFO_ID = "2"
CARRIER_ID = "3"
VIRTUAL_ID = "4"
CAMERA_DRONE_ID = "5"
MARKER_DRONE_ID = "6"

CATALOG_PATH = "../Catalogs/Vehicles"
CATALOG_NAME = "VehicleCatalog"
CONTROLLER_CATALOG_PATH = "../Catalogs/Controllers"
CONTROLLER_CATALOG_NAME = "ControllerCatalog"
MONDEO_CATALOG_NAME = "mondeo"
UFO_CATALOG_NAME = "humanetics"
CARRIER_CATALOG_NAME = "flexible_target_carrier"
VIRTUAL_CATALOG_NAME = "virtual_object"
CAMERA_DRONE_CATALOG_NAME = "camera_drone"
MARKER_DRONE_CATALOG_NAME = "marker_drone"

OPENDRIVE_PATH = "../odr/Multilane.xodr"
SCENARIO_NAME = "MultilaneScenario"
SCENARIO_FILE_NAME = SCENARIO_NAME + ".xosc"

In [3]:
# Dynamic parameters
ufo_speed = 10 / 3.6 # m/s
ufo_acceleration = 1.38 # m/s^2
ufo_retardation = 1.83 # m/s^2
ufo_O2_point = xosc.LanePosition(170, 0, 1, 0)
ufo_brake_position = xosc.LanePosition(50, 0, 1, 0)

carrier_speed = 15 / 3.6 # m/s
carrier_acceleration = 1.12 # m/s^2
carrier_retardation = 1.03 # m/s^2
carrier_O3_point = xosc.LanePosition(30, 0, -1, 0)
carrier_brake_position = xosc.LanePosition(130, 0, -1, 0)

camera_drone_speed = 15 / 3.6 # m/s
camera_drone_acceleration = 1.76 # m/s^2
camera_drone_retardation = 1.96 # m/s^2
camera_drone_O5_point = xosc.LanePosition(170, 2.5, 1, 0)
camera_drone_brake_position = xosc.LanePosition(50, 2.5, 1, 0)

denm_trigger_speed = 30 / 3.6 # m/s

carrier_trigger_positions = {
	UFO_ID: xosc.LanePosition(90, 0, -1, 0),
	CAMERA_DRONE_ID: xosc.LanePosition(90, 0, -1, 0),
}

In [4]:
# Preamble
init = xosc.Init()
osc_params = xosc.ParameterDeclarations()
osc_catalogs = xosc.Catalog()
osc_catalogs.add_catalog(CATALOG_NAME, CATALOG_PATH)
osc_catalogs.add_catalog(CONTROLLER_CATALOG_NAME, CONTROLLER_CATALOG_PATH)
osc_road_network = xosc.RoadNetwork(roadfile=OPENDRIVE_PATH)
osc_entities = xosc.Entities()
osc_storyboard = xosc.StoryBoard(init)
osc_act = xosc.Act("start")

ufo_maneuver_group = xosc.ManeuverGroup(UFO_ID + ",maneuver_group")
ufo_maneuver = xosc.Maneuver(UFO_ID + ",maneuver")
ufo_maneuver_group.add_actor(UFO_ID)
carrier_maneuver_group = xosc.ManeuverGroup(CARRIER_ID + ",maneuver_group")
carrier_maneuver = xosc.Maneuver(CARRIER_ID + ",maneuver")
carrier_maneuver_group.add_actor(CARRIER_ID)
camera_drone_maneuver_group = xosc.ManeuverGroup(CAMERA_DRONE_ID + ",maneuver_group")
camera_drone_maneuver = xosc.Maneuver(CAMERA_DRONE_ID + ",maneuver")
camera_drone_maneuver_group.add_actor(CAMERA_DRONE_ID)

In [5]:
osc_entities.add_scenario_object(UFO_ID,
	xosc.CatalogReference(CATALOG_NAME, UFO_CATALOG_NAME),
	xosc.CatalogReference(CONTROLLER_CATALOG_NAME, "externalController"))
osc_entities.add_scenario_object(CARRIER_ID,
	xosc.CatalogReference(CATALOG_NAME, CARRIER_CATALOG_NAME),
	xosc.CatalogReference(CONTROLLER_CATALOG_NAME, "externalController"))
osc_entities.add_scenario_object(CAMERA_DRONE_ID,
	xosc.CatalogReference(CATALOG_NAME, CAMERA_DRONE_CATALOG_NAME),
	xosc.CatalogReference(CONTROLLER_CATALOG_NAME, "externalController"))

ufo_start_teleport = xosc.TeleportAction(ufo_O2_point)
carrier_start_teleport = xosc.TeleportAction(carrier_O3_point)
camera_drone_start_teleport = xosc.TeleportAction(camera_drone_O5_point)

init.add_init_action(UFO_ID, ufo_start_teleport)
init.add_init_action(CARRIER_ID, carrier_start_teleport)
init.add_init_action(CAMERA_DRONE_ID, camera_drone_start_teleport)

In [6]:
# UFO movement profile:
# accelerate to top speed,
# follow a trajectory,
# then decelerate to a stop
times = []
positions = [
	xosc.LanePosition(170, 0, 1, 0),
	xosc.LanePosition(160, 0, 1, 0),
	xosc.LanePosition(150, 0, 1, 0),
	xosc.LanePosition(140, 0, 1, 0),
	xosc.LanePosition(130, 0, 1, 0),
	xosc.LanePosition(120, 0, 1, 0),
	xosc.LanePosition(100, 0, 1, 0),
	xosc.LanePosition(90, 0, 1, 0),
	xosc.LanePosition(80, 0, 1, 0),
	xosc.LanePosition(70, 0, 1, 0),
	xosc.LanePosition(60, 0, 1, 0),
	xosc.LanePosition(50, 0, 1, 0),
	xosc.LanePosition(40, 0, 1, 0),
	xosc.LanePosition(30, 0, 1, 0),
]
polyline = xosc.Polyline(times, positions)
trajectory = xosc.Trajectory(UFO_ID + ",start_follow_trajectory", closed=False)
trajectory.add_shape(polyline)
ufo_follow_trajectory = xosc.FollowTrajectoryAction(
	trajectory,
	xosc.FollowingMode.position,
	xosc.ReferenceContext.relative,
	1,
	0
)
ufo_set_speed = xosc.AbsoluteSpeedAction(
	ufo_speed,
	xosc.TransitionDynamics(
		xosc.DynamicsShapes.linear,
		xosc.DynamicsDimension.rate,
		ufo_acceleration
	),
)

ufo_brake = xosc.AbsoluteSpeedAction(
	0,
	xosc.TransitionDynamics(
		xosc.DynamicsShapes.linear,
		xosc.DynamicsDimension.rate,
		-ufo_retardation
	),
)
ufo_reach_stop_position = xosc.EntityTrigger(
	name=UFO_ID + ",reach_stop_position",
	delay=0,
	conditionedge=xosc.ConditionEdge.none,
	entitycondition=xosc.ReachPositionCondition(ufo_brake_position, 1),
	triggerentity=UFO_ID
)
ufo_brake_event = xosc.Event(
	UFO_ID + ",brake_event",
	xosc.Priority.parallel
)
ufo_brake_event.add_trigger(ufo_reach_stop_position)
ufo_brake_event.add_action(UFO_ID + ",brake_to_stop", ufo_brake)
ufo_maneuver.add_event(ufo_brake_event)

In [7]:
# Carrier movement profile:
# accelerate to top speed,
# follow a trajectory,
# then decelerate to a stop
times = []
positions = [
	xosc.LanePosition(30, 0, -1, 0),
	xosc.LanePosition(50, 0, -1, 0),
	xosc.LanePosition(70, 0, -1, 0),
	xosc.LanePosition(90, 0, -1, 0),
	xosc.LanePosition(110, 0, -1, 0),
	xosc.LanePosition(130, 0, -1, 0),
	xosc.LanePosition(150, 0, -1, 0),
	xosc.LanePosition(170, 0, -1, 0),
]
polyline = xosc.Polyline(times, positions)
trajectory = xosc.Trajectory(CARRIER_ID + ",start_follow_trajectory", closed=False)
trajectory.add_shape(polyline)
carrier_follow_trajectory = xosc.FollowTrajectoryAction(
	trajectory,
	xosc.FollowingMode.position,
	xosc.ReferenceContext.relative,
	1,
	0
)
carrier_scenario_started =  xosc.ValueTrigger(
	CARRIER_ID + ",scenario_started",
	0,
	xosc.ConditionEdge.none,
	xosc.SimulationTimeCondition(0, xosc.Rule.greaterThan)
)
carrier_follow_trajectory_event = xosc.Event(
	CARRIER_ID + ",carrier_follow_trajectory_event",
	xosc.Priority.parallel
)
carrier_set_speed = xosc.AbsoluteSpeedAction(
	carrier_speed,
	xosc.TransitionDynamics(
		xosc.DynamicsShapes.linear,
		xosc.DynamicsDimension.rate,
		carrier_acceleration
	),
)
carrier_follow_trajectory_event.add_trigger(carrier_scenario_started)
carrier_follow_trajectory_event.add_action(CARRIER_ID + ",init_follow_trajectory", carrier_follow_trajectory)
carrier_follow_trajectory_event.add_action(CARRIER_ID + ",set_speed", carrier_set_speed)
carrier_maneuver.add_event(carrier_follow_trajectory_event)

carrier_brake = xosc.AbsoluteSpeedAction(
	0,
	xosc.TransitionDynamics(
		xosc.DynamicsShapes.linear,
		xosc.DynamicsDimension.rate,
		-carrier_retardation
	),
)
carrier_reach_stop_position = xosc.EntityTrigger(
	name=CARRIER_ID + ",reach_stop_position",
	delay=0,
	conditionedge=xosc.ConditionEdge.none,
	entitycondition=xosc.ReachPositionCondition(carrier_brake_position, 1),
	triggerentity=CARRIER_ID
)
carrier_brake_event = xosc.Event(
	CARRIER_ID + ",brake_event",
	xosc.Priority.parallel
)
carrier_brake_event.add_trigger(carrier_reach_stop_position)
carrier_brake_event.add_action(CARRIER_ID + ",brake_to_stop", carrier_brake)
carrier_maneuver.add_event(carrier_brake_event)

In [8]:
# Camera drone movement profile:
# accelerate to top speed,
# follow a trajectory,
# then decelerate to a stop
times = []
positions = [
        xosc.LanePosition(170, 2.5, 1, 0),
        xosc.LanePosition(160, 2.5, 1, 0),
        xosc.LanePosition(150, 2.5, 1, 0),
        xosc.LanePosition(140, 2.5, 1, 0),
        xosc.LanePosition(130, 2.5, 1, 0),
        xosc.LanePosition(120, 2.5, 1, 0),
        xosc.LanePosition(100, 2.5, 1, 0),
        xosc.LanePosition(90, 2.5, 1, 0),
        xosc.LanePosition(80, 2.5, 1, 0),
        xosc.LanePosition(70, 2.5, 1, 0),
        xosc.LanePosition(60, 2.5, 1, 0),
        xosc.LanePosition(50, 2.5, 1, 0),
        xosc.LanePosition(40, 2.5, 1, 0),
        xosc.LanePosition(30, 2.5, 1, 0),
]
polyline = xosc.Polyline(times, positions)
camera_drone_trajectory = xosc.Trajectory(CAMERA_DRONE_ID + ",start_follow_trajectory", closed=False)
camera_drone_trajectory.add_shape(polyline)
camera_drone_follow_trajectory = xosc.FollowTrajectoryAction(
	camera_drone_trajectory,
	xosc.FollowingMode.position,
	xosc.ReferenceContext.relative,
	1,
	0
)
camera_drone_set_speed = xosc.AbsoluteSpeedAction(
	camera_drone_speed,
	xosc.TransitionDynamics(
		xosc.DynamicsShapes.linear,
		xosc.DynamicsDimension.rate,
		camera_drone_acceleration
	),
)
camera_drone_brake = xosc.AbsoluteSpeedAction(
	0,
	xosc.TransitionDynamics(
		xosc.DynamicsShapes.linear,
		xosc.DynamicsDimension.rate,
		-camera_drone_retardation
	),
)
camera_drone_reach_stop_position = xosc.EntityTrigger(
	name=CAMERA_DRONE_ID + ",reach_stop_position",
	delay=0,
	conditionedge=xosc.ConditionEdge.none,
	entitycondition=xosc.ReachPositionCondition(camera_drone_brake_position, 1),
	triggerentity=CAMERA_DRONE_ID
)
camera_drone_brake_event = xosc.Event(
	CAMERA_DRONE_ID + ",brake_event",
	xosc.Priority.parallel
)
camera_drone_brake_event.add_trigger(camera_drone_reach_stop_position)
camera_drone_brake_event.add_action(CAMERA_DRONE_ID + ",brake_to_stop", camera_drone_brake)
camera_drone_maneuver.add_event(camera_drone_brake_event)

In [9]:
# Carrier triggers UFO
carrier_reached_ufo_trigger_position = xosc.EntityTrigger(
	name=CARRIER_ID + ",reached_ufo_trigger_position",
	delay=0,
	conditionedge=xosc.ConditionEdge.none,
	entitycondition=xosc.ReachPositionCondition(carrier_trigger_positions[UFO_ID], 1),
	triggerentity=CARRIER_ID
)

ufo_move_event = xosc.Event(
	UFO_ID + ",start_move_event",
	xosc.Priority.parallel
)
ufo_move_event.add_trigger(carrier_reached_ufo_trigger_position)
ufo_move_event.add_action(UFO_ID + ",start_follow_trajectory", ufo_follow_trajectory)
ufo_move_event.add_action(UFO_ID + ",set_speed", ufo_set_speed)
ufo_maneuver.add_event(ufo_move_event)

In [10]:
# Carrier triggers camera drone
carrier_reached_camera_drone_trigger_position = xosc.EntityTrigger(
	name=CARRIER_ID + ",reached_camera_drone_trigger_position",
	delay=0,
	conditionedge=xosc.ConditionEdge.none,
	entitycondition=xosc.ReachPositionCondition(carrier_trigger_positions[CAMERA_DRONE_ID], 1),
	triggerentity=CARRIER_ID
)

camera_drone_move_event = xosc.Event(
	CAMERA_DRONE_ID + ",start_move_event",
	xosc.Priority.parallel
)
camera_drone_move_event.add_trigger(carrier_reached_camera_drone_trigger_position)
camera_drone_move_event.add_action(CAMERA_DRONE_ID + ",start_follow_trajectory", camera_drone_follow_trajectory)
camera_drone_move_event.add_action(CAMERA_DRONE_ID + ",set_speed", camera_drone_set_speed)
camera_drone_maneuver.add_event(camera_drone_move_event)

In [11]:
# Carrier object triggers DENM warning

### DENM Action ###
carrier_high_speed_detected = xosc.EntityTrigger(
	name=CARRIER_ID + ",high_speed_detected",
	delay=0,
	conditionedge=xosc.ConditionEdge.none,
	entitycondition=xosc.SpeedCondition(denm_trigger_speed, xosc.Rule.greaterThan),
	triggerentity=CARRIER_ID)

carrier_denm_action = xosc.VisibilityAction(graphics=True, traffic=True, sensors=True)

denm_osc_event = xosc.Event(
	CARRIER_ID + ",high_speed_event",
	xosc.Priority.parallel,
)

denm_osc_event.add_trigger(carrier_high_speed_detected)
denm_osc_event.add_action(CARRIER_ID + ",send_denm",
							carrier_denm_action)
carrier_maneuver.add_event(denm_osc_event)

In [12]:
# Collect into a scenario and write to file
ufo_maneuver_group.add_maneuver(ufo_maneuver)
carrier_maneuver_group.add_maneuver(carrier_maneuver)
camera_drone_maneuver_group.add_maneuver(camera_drone_maneuver)

osc_act.add_maneuver_group(ufo_maneuver_group)
osc_act.add_maneuver_group(carrier_maneuver_group)
osc_act.add_maneuver_group(camera_drone_maneuver_group)

osc_storyboard.add_act(osc_act)
osc_file = xosc.Scenario(
		name=SCENARIO_NAME,
		author="AstaZero 1.0",
		parameters=osc_params,
		entities=osc_entities,
		storyboard=osc_storyboard,
		roadnetwork=osc_road_network,
		catalog=osc_catalogs
	)

osc_file.write_xml(SCENARIO_FILE_NAME)